## Setup

In [1]:
!pip3 install --upgrade pip -q
!pip3 install nvidia-cublas-cu12==12.4.5.8 nvidia-cuda-cupti-cu12==12.4.127 nvidia-cuda-nvrtc-cu12==12.4.127 nvidia-cuda-runtime-cu12==12.4.127 nvidia-cudnn-cu12==9.1.0.70 nvidia-cufft-cu12==11.2.1.3 nvidia-curand-cu12==10.3.5.147 nvidia-cusolver-cu12==11.6.1.9 nvidia-cusparse-cu12==12.3.1.170 nvidia-nvjitlink-cu12==12.4.127
!pip3 install nltk emoji==0.6.0 -q
!pip3 install torchinfo -q
!pip3 install kagglehub -q
!pip3 install kaggle -q
!pip3 install transformers -q
!pip3 install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 137.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 119.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unin

In [2]:
!pip3 install fsspec==2025.3.0

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# list the elements in the current working directory
print("current directory: ", os.getcwd())
#os.chdir('content/drive/MyDrive/DATASCI 266/Project') # Use when working on Berkeley's Google Drive
os.chdir('drive/MyDrive/Project') # Use when working on Personal Google Drive
os.listdir()

current directory:  /content


['Project Proposal.gdoc',
 'TweetNormalizer.py',
 '__pycache__',
 'results_baseline',
 'wandb',
 'Tweets_full_normalized.csv',
 'baseline_model',
 'bertweet_model',
 'bertkeep_model',
 'Copy of Lora_(Roberta_Large)_pytorch.ipynb',
 'filtered_tweets.csv',
 'filtered_sample_2500.csv',
 'Normalized_filtered_tweets.csv',
 'Crypto Sentiment Analysis.ipynb',
 'labelled_normalized_dataset.csv',
 'small_train_dataset.csv',
 'small_eval_dataset.csv',
 'small_eval_dataset_filtered.csv',
 'small_train_dataset_filtered.csv',
 'labelled_dataset.csv',
 'bertkeep_model5',
 'bertweet_model_6',
 'bertweet_model_7',
 'bertweet_model_8',
 'bertweet_model_9',
 'bertkeep_model_9',
 'bertkeep_model_10',
 'bertkeep_model_11',
 'Paper',
 'tweets_sentiment.csv',
 'File Index.gsheet',
 'Crypto Sentiment Analysis_Data Preparation.ipynb',
 'Copy of Crypto Sentiment Analysis.ipynb',
 'Crypto Sentiment Analysis_BERTweet.ipynb',
 'large_balanced_dataset.csv',
 'Untitled0.ipynb',
 'Model_Test_MG.ipynb',
 'Copy of Cry

In [5]:
import kagglehub
import pandas as pd
import numpy as np

import torch
from torchinfo import summary # to show the summary of the models
import evaluate # for metric evaluation during training from Huggingface
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from TweetNormalizer import normalizeTweet
from datasets import Dataset # from Huggingface

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType


## Data Analysis

Data Source: Kaggle  
Location in Kaggle: fabioturazzi/cryptocurrency-tweets-with-sentiment-analysis  
File name: tweets_sentiment.csv    
Link: https://www.kaggle.com/api/v1/datasets/download/fabioturazzi/cryptocurrency-tweets-with-sentiment-analysis?dataset_version_number=1  

In [ ]:
sample_csv_path = '/content/drive/.shortcut-targets-by-id/1xT6cdCXD4s5pURHY0Jd8BkLnI-Qn4Hyy/Project'


In [ ]:
file_name = "filtered_tweets.csv"
df = pd.read_csv(sample_csv_path + "/" + file_name)

In [ ]:
print(df.shape)
df.info()
print(f"Date range: {df.date.min()} - {df.date.max()}")


In [ ]:
# Update the path as needed — this uses your previous working directory
save_path = '/content/drive/MyDrive/Project/Normalized_filtered_tweets.csv'

# Save to CSV
#df2.to_csv(save_path, index=False)

print(f"Saved to: {save_path}")

## BERTweet

BERTweet Model: [Huggingface BERTweet](https://huggingface.co/docs/transformers/en/model_doc/bertweet)  
BERTweet Paper: [BERTweet: A pre-trained language model for English Tweets](https://aclanthology.org/2020.emnlp-demos.2.pdf)


### Data Preparation

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Project/labelled_dataset.csv')

In [ ]:
# Normalize the text on the tweets


df2.tweet = df2.tweet.apply(normalizeTweet)

In [ ]:
# Check that the tweets are normalized
df2.iloc[563793:569740]

,date,tweet,neg,neu,pos,compound,tweet_clean,label
563793,2019-12-15,The design of digital financial infrastructure...,0.0,0.909,0.091,0.3182,The design of digital financial infrastructure...,0
563794,2019-12-16,پیامرسان ها درآمد سرشاری دارند اما بزودی به هم...,0.0,0.886,0.114,0.5972,پیامرسان ها درآمد سرشاری دارند اما بزودی به هم...,0
563795,2020-10-04,Hi 25 cent :beaming_face_with_smiling_eyes:,0.0,0.727,0.273,0.4588,Hi 25 cent 😁,0
563796,2020-10-27,$ ocean $ Ocean $ ewt $ dot $ qnt $ xrp,0.0,1.000,0.000,0.0000,$ocean $Ocean $ewt $dot $qnt $xrp,0
563797,2020-11-21,Hi . 42 cent :sign_of_the_horns: :handshake: :...,0.0,0.842,0.158,0.4939,Hi .42 cent 🤘🤝🤙✅🍺,0
563798,2021-01-27,An entire continent ...,0.0,1.000,0.000,0.0000,An entire continent...,0


In [ ]:
# Analyze tweet length

pd.DataFrame([len(i) for i in df2.tweet]).describe(percentiles=[.9,.95,.99])


,0
count,563799.000000
mean,141.214032
std,79.614512
min,4.000000
50%,129.000000
90%,258.000000
95%,279.000000
99%,311.000000
max,1611.000000


In [ ]:
# Look at number of labels

df2['label'].value_counts()


,count
label,
0,352018
2,157764
1,54017


In [8]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Get a small sample with the same number of each label for tunning.

max_length =  280
sample_size = 5000

# Take the Sample Using Sentiment
df2_sample = pd.concat(
    [df2.loc[df2.label==x,['tweet','label']].sample(sample_size) for x in df2.label.unique()])
small_train_dataset, small_eval_dataset = train_test_split(df2_sample, test_size=0.2)

# Preprocess your datasets


small_train_dataset.to_csv("small_train_dataset_filtered.csv")
small_eval_dataset.to_csv("small_eval_dataset_filtered.csv")


In [9]:
# Load training data
#df2 = pd.read_csv("Tweets_full_normalized.csv")
small_train_dataset = pd.read_csv("small_train_dataset_filtered.csv")
small_eval_dataset = pd.read_csv("small_eval_dataset_filtered.csv")

In [ ]:
small_train_dataset.shape

(12000, 3)

### BERTweet Baseline Classifier

In [13]:


#small_train_dataset['labels'] = small_train_dataset['labels'].map(label_to_id)
#small_eval_dataset['labels'] = small_eval_dataset['labels'].map(label_to_id)

small_train_dataset = Dataset.from_pandas(small_train_dataset, preserve_index=False)
small_eval_dataset = Dataset.from_pandas(small_eval_dataset, preserve_index=False)

max_length =  280


# Define the preprocess function
def preprocess_tweet(data):
    review_text = data['tweet']
    encoded = tokenizer.batch_encode_plus(
            review_text,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )
    encoded['label'] = torch.tensor(data['label'])
    return encoded

# Uncomment these lines so they are applied to the data:
small_train_dataset = small_train_dataset.map(preprocess_tweet, batched=True)
small_eval_dataset = small_eval_dataset.map(preprocess_tweet, batched=True)

AttributeError: 'Dataset' object has no attribute 'columns'

In [12]:
bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-large",
                                                              problem_type="single_label_classification",
                                                              num_labels = 3)

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Freezing the pretrained layers

In [ ]:
for name, param in bertweet.named_parameters():
    print(name)

roberta.embeddings.word_embeddings.weight
roberta.embeddings.position_embeddings.weight
roberta.embeddings.token_type_embeddings.weight
roberta.embeddings.LayerNorm.weight
roberta.embeddings.LayerNorm.bias
roberta.encoder.layer.0.attention.self.query.weight
roberta.encoder.layer.0.attention.self.query.bias
roberta.encoder.layer.0.attention.self.key.weight
roberta.encoder.layer.0.attention.self.key.bias
roberta.encoder.layer.0.attention.self.value.weight
roberta.encoder.layer.0.attention.self.value.bias
roberta.encoder.layer.0.attention.output.dense.weight
roberta.encoder.layer.0.attention.output.dense.bias
roberta.encoder.layer.0.attention.output.LayerNorm.weight
roberta.encoder.layer.0.attention.output.LayerNorm.bias
roberta.encoder.layer.0.intermediate.dense.weight
roberta.encoder.layer.0.intermediate.dense.bias
roberta.encoder.layer.0.output.dense.weight
roberta.encoder.layer.0.output.dense.bias
roberta.encoder.layer.0.output.LayerNorm.weight
roberta.encoder.layer.0.output.LayerNorm

In [ ]:
for name, param in bertweet.named_parameters():
    if name.split(".")[0] == "roberta":
        param.requires_grad = False

print("Layers to train:\n")
for name, param in bertweet.named_parameters():
    if param.requires_grad :
      print(name)
print("\nBaseline model Structure\n")
summary(bertweet)

In [ ]:
# Create Training arguments
training_args = TrainingArguments(
    output_dir='./results_baseline',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    report_to=None  # Disable wandb logging
)

metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

#Training
trainer = Trainer(
    model=bertweet,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
output_dir = "./baseline_model"
trainer.save_model(output_dir)

## LoRA Fine Tunning


In [14]:
# Read the baseline model from the folder
output_dir = "/content/drive/MyDrive/Project/baseline_model"
baseline_model = AutoModelForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)


In [15]:
small_train_dataset = pd.read_csv("small_train_dataset_filtered.csv")
small_eval_dataset = pd.read_csv("small_eval_dataset_filtered.csv")
#small_train_dataset.head(10)

In [16]:
# Load training data
#df2 = pd.read_csv("Tweets_full_normalized.csv")
max_length =  280
small_train_dataset = pd.read_csv("small_train_dataset_filtered.csv")
small_eval_dataset = pd.read_csv("small_eval_dataset_filtered.csv")
small_train_dataset = Dataset.from_pandas(small_train_dataset, preserve_index=False)
small_eval_dataset = Dataset.from_pandas(small_eval_dataset, preserve_index=False)
# Define the preprocess function
def preprocess_tweet(data):
    review_text = data['tweet']
    encoded = tokenizer.batch_encode_plus(
            review_text,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )
    encoded['label'] = torch.tensor(data['label'])
    return encoded

small_train_dataset = small_train_dataset.map(preprocess_tweet, batched=True)
small_eval_dataset = small_eval_dataset.map(preprocess_tweet, batched=True)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [17]:
# LoRA Parameters

r = 4
lora_alpha = 512
lora_dropout = 0.1

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout
)
bertweet_model = get_peft_model(baseline_model, peft_config)
bertweet_model.print_trainable_parameters()

trainable params: 1,445,891 || all params: 356,808,710 || trainable%: 0.4052


In [31]:
_= bertweet_model.to("cuda")

In [32]:
lr = 1e-5
batch_size = 16
num_epochs = 5
weight_decay = 0.01

training_args = TrainingArguments(
    output_dir="./bertkeep_model_11",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
)

metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
trainer = Trainer(
    model=bertweet_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,


)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.503700,0.551577,0.803333
2,0.466100,0.492102,0.830000
3,0.335600,0.487369,0.836667
4,0.488700,0.477390,0.836000
5,0.454300,0.488118,0.836667


TrainOutput(global_step=3750, training_loss=0.4457461725234985, metrics={'train_runtime': 1710.6885, 'train_samples_per_second': 35.074, 'train_steps_per_second': 2.192, 'total_flos': 3.0724846992e+16, 'train_loss': 0.4457461725234985, 'epoch': 5.0})

In [34]:
output_dir = "./bertkeep_model_11"

trainer.save_model(output_dir)

In [25]:
model = bertweet_model  # use the LoRA-fine-tuned model


# Load and convert your large CSV (if not done yet)
df_large = pd.read_csv("large_balanced_dataset.csv")
dataset_large = Dataset.from_pandas(df_large, preserve_index=False)

# Tokenize the text
def preprocess_tweet(data):
    encoded = tokenizer.batch_encode_plus(
        data['tweet'],
        max_length=280,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors="pt"
    )
    encoded['label'] = torch.tensor(data['label'])
    return encoded

dataset_large = dataset_large.map(preprocess_tweet, batched=True)
dataset_large.set_format(type='torch')


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [26]:
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from tqdm import tqdm

dataloader = DataLoader(dataset_large, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


100%|██████████| 3125/3125 [18:05<00:00,  2.88it/s]


In [27]:
from sklearn.metrics import classification_report

print(classification_report(all_labels, all_preds))


              precision    recall  f1-score   support

           0       0.96      0.69      0.81     62437
           1       0.43      0.88      0.58      9581
           2       0.66      0.84      0.74     27982

    accuracy                           0.75    100000
   macro avg       0.69      0.80      0.71    100000
weighted avg       0.83      0.75      0.77    100000

